In [1]:
using DifferentialEquations, GLMakie, StaticArrays

In [2]:
using DataStructures: CircularBuffer

In [4]:
θ1 = π/2
θ2 = 0
global m1 = 1
global m2 = 1
global L1 = 3
global L2 = 1
x0 = 0
y0 = 0
global g = 9.81
Δ = 0.01
tfinal = 200
tail = 100

100

In [5]:
function ω1_prim(θ1, θ2, ω1, ω2)
    comp1 = -g*(2*m1 + m2)*sin(θ1)
    comp2 = -m2*g*sin(θ1 - 2*θ2)
    comp3 = -2*sin(θ1 - θ2)*m2*(ω2*ω2*L2 + ω1*ω1*L1*cos(θ1 - θ2))
    denom = L1*(2*m1 + m2 - m2*cos(2*θ1 - 2*θ2))
    
    (comp1 + comp2 + comp3) / denom
end

ω1_prim (generic function with 1 method)

In [6]:
function ω2_prim(θ1, θ2, ω1, ω2)
    num = 2*sin(θ1 − θ2)*(ω1*ω1*L1*(m1 + m2) + g*(m1 + m2)*cos(θ1) + ω2*ω2*L2*m2*cos(θ1 − θ2))
    denom = L2*(2*m1 + m2 − m2*cos(2*θ1 − 2*θ2))
            
    num / denom
end

ω2_prim (generic function with 1 method)

In [7]:
function solve_it!(du, u, p, t)    
    θ1 = u[1]
    θ2 = u[3]
    ω1 = u[2]
    ω2 = u[4]
    
    du[1] = u[2]
    du[2] = ω1_prim(θ1, θ2, ω1, ω2)
    du[3] = u[4]
    du[4] = ω2_prim(θ1, θ2, ω1, ω2)
end

solve_it! (generic function with 1 method)

In [8]:
function double_pendulum(θ1, θ2, x0, y0, Δ, tfinal)
    u0 = [θ1; 0; θ2; 0]
    p = [m1, m2, L1, L2, g]
    tspan = (0.0, tfinal)
    prob = ODEProblem(solve_it!, u0, tspan, p)
    integ = init(prob, Vern7(), reltol=1e-6, saveat=Δ)
end

double_pendulum (generic function with 1 method)

In [9]:
function xycords(state)
    θ1 = state[1]
    θ2 = state[3]
    x1 = L1 * sin(θ1)
    y1 = -L1 * cos(θ1)
    x2 = x1 + L2 * sin(θ2)
    y2 = y1 - L2 * cos(θ2)
    return x1,x2,y1,y2
end

xycords (generic function with 1 method)

In [10]:
function progress_for_one_step!(integ)
    step!(integ)
    u = integ.u
    return xycords(u)
end

progress_for_one_step! (generic function with 1 method)

In [11]:
function makefig(θ1, θ2, x0, y0, Δ, tfinal)
    integ = double_pendulum(θ1, θ2, x0, y0, Δ, tfinal)
    x1,x2,y1,y2 = xycords(integ.u)
    rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
    balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
    traj = CircularBuffer{Point2f}(tail)
    fill!(traj, Point2f(x2, y2))
    traj = Observable(traj)
    fig = Figure(resolution = (1200, 800)); display(fig)
    ax = Axis(fig[1,1])
    lines!(ax, rod; linewidth = 4, color = :purple)
    scatter!(ax, balls; marker = :circle, strokewidth = 2, 
        strokecolor = :purple,
        color = :black, markersize = [8, 12]
    )
    c = to_color(:purple)
    tailcol = [RGBAf(c.r, c.g, c.b, (i/tail)^2) for i in 1:tail]
    lines!(ax, traj; linewidth = 3, color = tailcol)
    ax.title = "double pendulum"
    ax.aspect = DataAspect()
    l = 10
    xlims!(ax, -l, l)
    ylims!(ax, -l, 0.5l)
    return fig, integ, rod, balls, traj
end

makefig (generic function with 1 method)

In [12]:
function animstep!(integ, rod, balls, traj)
    x1,x2,y1,y2 = progress_for_one_step!(integ)
    rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    push!(traj[], Point2f(x2, y2))
    traj[] = traj[]
end

animstep! (generic function with 1 method)

In [13]:
function add_run()
    run = Button(fig[2,1]; label = "run", tellwidth = false)
    isrunning = Observable(false)
    on(run.clicks) do clicks; isrunning[] = !isrunning[]; end
    on(run.clicks) do clicks
        @async while isrunning[]
            isopen(fig.scene) || break
            animstep!(integ, rod, balls, traj)
            sleep(0.01)
        end
    end
end

add_run (generic function with 1 method)

In [14]:
function add_sliders()
    lsgrid = labelslidergrid!(
        fig,
        ["m1", "m2", "L1", "L2", "g"],
        [0:0.1:30, 0:0.1:30, 0:0.1:5, 0:0.1:5, 0:0.1:30];
        formats = [x -> "$(round(x, digits = 1))$s" for s in ["kg", "kg", "m", "m", "m/s^2"]],
        width = 350,
        tellheight = false)
    fig[1, 2] = lsgrid.layout

    slider_observables = [s.value for s in lsgrid.sliders]
    set_close_to!(lsgrid.sliders[1], m1)
    on(slider_observables[1]) do val
        global m1 = val
        reinit!(integ, [θ1; 0; θ2; 0])
        x1,x2,y1,y2 = xycords(integ.u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end

    set_close_to!(lsgrid.sliders[2], m2)
    on(slider_observables[2]) do val
        global m2 = val
        reinit!(integ, [θ1; 0; θ2; 0])
        x1,x2,y1,y2 = xycords(integ.u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end

    set_close_to!(lsgrid.sliders[3], L1)
    on(slider_observables[3]) do val
        global L1 = val
        reinit!(integ, [θ1; 0; θ2; 0])
        x1,x2,y1,y2 = xycords(integ.u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end

    set_close_to!(lsgrid.sliders[4], L2)
    on(slider_observables[4]) do val
        global L2 = val
        reinit!(integ, [θ1; 0; θ2; 0])
        x1,x2,y1,y2 = xycords(integ.u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end

    set_close_to!(lsgrid.sliders[5], g)
    on(slider_observables[5]) do val
        global g = val
        reinit!(integ, [θ1; 0; θ2; 0])
        x1,x2,y1,y2 = xycords(integ.u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end
end

add_sliders (generic function with 1 method)

In [15]:
function add_click()
    ax = content(fig[1,1])
    Makie.deactivate_interaction!(ax, :rectanglezoom)
    spoint = select_point(ax.scene)
    function θωcords(x, y)
        θ = atan(y,x) + π/2
        return SVector(θ,0,0,0)
    end

    on(spoint) do z
        x, y = z
        u = θωcords(x, y)
        reinit!(integ, u)
        x1,x2,y1,y2 = xycords(u)
        traj[] .= fill(Point2f(x2, y2), length(traj[]))
        traj[] = traj[]
        rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
        balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    end
end

add_click (generic function with 1 method)

In [16]:
global m1 = 15
global m2 = 15
global L1 = 3
global L2 = 3
global g = 9.8
fig, integ, rod, balls, traj = makefig(θ1, θ2, x0, y0, Δ, tfinal)
add_run()
add_sliders()
add_click()

(::Observables.ObserverFunction) (generic function with 0 methods)

In [17]:
# global m1 = 15
# global m2 = 15
# global L1 = 3
# global L2 = 3
# global g = 9.8
# fig, integ, rod, balls, traj = makefig(θ1, θ2, x0, y0, Δ, tfinal)
# add_run()
# add_sliders()
# add_click()
# frames = 1:3000
# record(fig, "video.mp4", frames; framerate = 30) do i
# end 

In [18]:
display("text/html", """
    <video alt="test" controls>
        <source src="video.mp4" type="video/mp4" 
    </video>""")